In [1]:
from ortools.sat.python import cp_model

In [2]:
def CodeSample():
    model = cp_model.CpModel()
    x = model.NewBoolVar('x')
    print(x)

In [3]:
CodeSample()

x


In [4]:
def LiteralSample():
    model = cp_model.CpModel()
    x = model.NewBoolVar('x')
    not_x = x.Not()
    print(x)
    print(not_x)

In [5]:
LiteralSample()

x
not(x)


In [12]:
# Boolean constraints
def BoolOrSample():
    model = cp_model.CpModel()
    x = model.NewBoolVar('x')
    y = model.NewBoolVar('y')
    model.AddBoolOr([x, y.Not()])

In [13]:
BoolOrSample()

In [14]:
# reified constraints
def ReifiedSample():
    """Showcase creating a reified constraint."""
    model = cp_model.CpModel()
    
    x = model.NewBoolVar('x')
    y = model.NewBoolVar('y')
    b = model.NewBoolVar('b')
    
    # First version using a half-reified bool and.
    model.AddBoolAnd([x, y.Not()]).OnlyEnforceIf(b)
    
    # Second version using implications.
    model.AddImplication(b, x)
    model.AddImplication(b, y.Not())
    
    # Third version using bool or.
    model.AddBoolOr([b.Not(), x])
    model.AddBoolOr([b.Not(), y.Not()])

In [15]:
ReifiedSample()

In [16]:
# Rabbits and Pheasants examples
def RabbitsAndPheasants():
    """Solves the rabbits + pheasants problem."""
    # 1. model
    model = cp_model.CpModel()
    # 2. variables
    r = model.NewIntVar(0, 100, 'r')
    p = model.NewIntVar(0, 100, 'p')
    
    # 3. constraints
    # 20 heads.
    model.Add(r + p == 20)
    # 56 legs.
    model.Add(4 * r + 2 * p == 56)
    
    # 4. solver
    # Solves and prints out the solution.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    # printer
    if status == cp_model.FEASIBLE:
        print('{} rabbits and {} pheasants'.format(solver.Value(r), solver.Value(p)))    
    

In [17]:
RabbitsAndPheasants()

8 rabbits and 12 pheasants


In [29]:
# Channeling constraints.
"""Link integer constraints together."""
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""
    def  __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        
    def OnSolutionCallback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()
    
    def SolutionCount(self):
        return self.__solution_count
    
def ChannelingSample():
    """Demonstrates how to link integer constraints together."""
    # Model.
    model = cp_model.CpModel()
    
    # Variables.
    x = model.NewIntVar(0, 10, 'x')
    y = model.NewIntVar(0, 10, 'y')
    
    b = model.NewBoolVar('b')
    
    # Implement b == (x >= 5).
    model.Add(x >= 5).OnlyEnforceIf(b)
    model.Add(x < 5).OnlyEnforceIf(b.Not())
    
    # b implies (y == 10 - x).
    model.Add(y == 10 - x).OnlyEnforceIf(b)
    # not(b) implies y == 0.
    model.Add(y == 0).OnlyEnforceIf(b.Not())
    
    # search for x values in increasing order.
    model.AddDecisionStrategy([x], cp_model.CHOOSE_FIRST,
                             cp_model.SELECT_MIN_VALUE)
    
    # create a solver and solve with a fixed search.
    solver = cp_model.CpSolver()
    
    # Force solver to follow the decision strategy exactly.
    # solver.parameters.search_branching = cp_model.
    
    # searches and prints out all solutions.
    solution_printer = VarArraySolutionPrinter([x, y, b])
    solver.SearchForAllSolutions(model, solution_printer)
    solution_printer.OnSolutionCallback()
    print(solution_printer.SolutionCount())

In [30]:
ChannelingSample()

x=5 y=5 b=1 
1


In [ ]:
# A bin-packing problem
import ortools